In [1]:
import numpy as np
import json
import os

In [2]:
# read multiplication table file
# Windows infile
#infile = [np.asarray(json.loads(line)) for line in open(os.path.abspath(os.pardir) + '\z2_s4_001.json', 'r')]
# OS infile
infile = [np.asarray(json.loads(line)) for line in open(os.path.abspath(os.pardir) + '/z2_s4_001.json', 'r')]
mixer_index = infile[0]
M = infile[1] #multipliers
T = infile[2] #multiplication table

assert T.shape[0] == T.shape[1] == T.shape[2]
len_T = T.shape[0]
print("Number of conjugacy classes: {}" .format(len_T))


assert M.shape[1] == len_T
len_M = M.shape[0]
print("Number of multipliers: {}" .format(len_M))

#print("Multiplier Indices: {}" .format(M))

Number of conjugacy classes: 33
Number of multipliers: 9


In [3]:
# Convert multplication table from 3D array of integers to 3D list of bin
def array_to_list(X):
    return X.tolist()

M = M.astype(int).tolist()
T = T.astype(int)

In [4]:
nbytes = 2
PAD = '0'

def pad_string(m, len_T, PAD):
    pad_length = (len_T - len(m) % len_T)
    pad = PAD*pad_length
    return m + pad, pad_length

def string_to_byte_sequence(m, nbytes):
    #convert string to binary sequence
    #print(bin_version)
    return ''.join(format(ord(i), '08b') for i in m)

def byte_sequence_chunk(m, nbytes):
    #seperate byte sequence into list of 'chunks' of length 'bits_per_coefficient'
    bits_per_coefficient = nbytes * 2
    res = []
    for start in range(0, len(m), bits_per_coefficient):
        res.append(m[start:start + bits_per_coefficient])
    return res

def byte_sequence_to_binary_sequence(m):
    #interpret each byte chunk as binary integer 
    return [int(e,2) for e in m]

def binary_block_sequence(m, len_T):
    #seperate chunks into blocks of length 'len_T'
    return [m[i:i + len_T] for i in range(0, len(m), len_T)]

def binary_block_sequence_to_byte_sequence(m, nbytes):
    res = ''
    chunk_size = nbytes*2
    for i in range(len(m)):
        for j in range(len(m[i])):
            byte = str(bin(m[i][j])[2:].zfill(chunk_size))
            res += byte
    return res

def byte_to_pad_string(m):
    return ''.join(chr(int(m[i*8:i*8+8],2)) for i in range(len(m)//8))

def unpad_string(m, pad_length):
    return m[:len(m)-pad_length]
    

In [5]:
def string_to_bring(m, len_T, nbytes = 2, PAD = '0'):
    m1, pad_length = pad_string(m, len_T, PAD)
    m2 = string_to_byte_sequence(m1, nbytes)
    m3 = byte_sequence_chunk(m2, nbytes)
    m4 = byte_sequence_to_binary_sequence(m3)
    m5 = binary_block_sequence(m4, len_T)
    return m5, pad_length

def bring_to_string(m, len_T, pad_length, nbytes = 2, PAD = '0'):
    m1 = binary_block_sequence_to_byte_sequence(m, nbytes)
    m2 = byte_to_pad_string(m1)
    m3 = unpad_string(m2, pad_length)
    return m3

In [39]:
# Demonstration
m = 'abcdefg'
print("String: %s; Length: %s" % (m, len(m)))
m1, pad_length = pad_string(m, len_T, PAD)
print("Padded String: %s; Length: %s" % (m1, len(m1)))
print("Pad Length: %s" % (pad_length))
m2 = string_to_byte_sequence(m1, nbytes)
print("Byte Sequence: %s; Length: %s" % (m2, len(m2)))
m3 = byte_sequence_chunk(m2, nbytes)
print("Byte Sequence Chunks: %s" % (m3))
m4 = byte_sequence_to_binary_sequence(m3)
print("Binary Sequence: %s" % (m4))
m5 = binary_block_sequence(m4, len_T)
print("Binary Sequence Blocks: %s" % (m5))
m6 = binary_block_sequence_to_byte_sequence(m5, nbytes)
print("Byte Sequence: %s; Length: %s" % (m6, len(m6)))
m7 = byte_to_pad_string(m6)
print("Reconstructed Padded String: %s" % (m7))
m8 = unpad_string(m7, pad_length)
print("Reconstructed String: %s" % (m8))

String: abcdefg; Length: 7
Padded String: abcdefg00000000000000000000000000; Length: 33
Pad Length: 26
Byte Sequence: 011000010110001001100011011001000110010101100110011001110011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000; Length: 264
Byte Sequence Chunks: ['0110', '0001', '0110', '0010', '0110', '0011', '0110', '0100', '0110', '0101', '0110', '0110', '0110', '0111', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000']
Binary Sequence: [6, 1, 6, 2, 6, 3, 6, 4, 6, 5, 6, 6,

In [9]:
def brmult_list(T, L, sel = "none"):
    #L: ring elements to be multiplied
    #T: multiplication table
    if sel == "none":
        pass
    else:
        L = L[sel,:]
        
    for l in L:
        if l == L[0]: # initialize product
            chi = l
        else:
            temp = np.zeros(len(l))
            for i in range(len(l)):
                for j in range(len(l)):
                    # chi[i]*l[j] is the appropriate coefficient
                    # T[i][j] is the appropriate ring element
                    temp+= chi[i]*l[j]*T[i][j]
            chi = temp
            #print("-------------------------------------------------------------")
            #print(chi) 
    return chi.astype(int).tolist()

In [10]:
msg = 'abcdefghijk'
bring_msg, pad_length = string_to_bring(msg, len_T)
print(bring_msg)

[[6, 1, 6, 2, 6, 3, 6, 4, 6, 5, 6, 6, 6, 7, 6, 8, 6, 9, 6, 10, 6, 11, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3], [0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0]]


In [11]:
multiplier = brmult_list(T,M)
print(multiplier)

[1, -1, 2, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, -1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -2, -2, -1, -1, -1, 1]


In [15]:
def bring_encryption(T,multiplier,m):
    #T: Multiplication table
    #multiplier: multiplier element in bring
    #m: bring representation of msg
    return [brmult_list(T,[e,multiplier]) for e in bring_msg]

In [16]:
encrypted_bring_msg = bring_encryption(T, multiplier, bring_msg)
print(bring_msg)
print(encrypted_bring_msg)

[[6, 1, 6, 2, 6, 3, 6, 4, 6, 5, 6, 6, 6, 7, 6, 8, 6, 9, 6, 10, 6, 11, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3], [0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0]]
[[12, -11, 20, -10, -24, -21, -9, -21, -9, -14, -26, -15, -12, 13, -23, 8, -3, -6, -3, -7, -3, 17, 9, 6, 0, 6, 0, -6, -9, -3, -6, -3, 3], [9, -9, 9, -6, -9, -12, 0, -9, 0, -6, -3, -6, -6, 9, -6, 3, 0, -3, 0, -3, 0, 3, 0, 3, 0, 3, 0, -3, 0, -3, 0, -3, 0]]


In [19]:
decrypted_bring_msg = [brmult_list(T,[e,multiplier]) for e in encrypted_bring_msg]
print(decrypted_bring_msg)


[[6, 1, 6, 2, 6, 3, 6, 4, 6, 5, 6, 6, 6, 7, 6, 8, 6, 9, 6, 10, 6, 11, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3], [0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0]]


In [21]:
reconstructed_msg = bring_to_string(decrypted_bring_msg, len_T, pad_length)
print(reconstructed_msg)

abcdefghijk
